In [ ]:
import pandas as pd
from textblob import TextBlob
from itertools import combinations

In [44]:
# Define the multiple-choice questions
questions = [
    "How are you feeling today?",
    "Did you enjoy your day?",
    "What is your favorite color?",
    "Do you prefer tea or coffee?",
    "Which movie genres do you enjoy? (Select multiple options)"
]

# Define the answer options for each question
answers = [
    # Question 1
    ["Great", "Good", "Neutral", "Not so good", "Terrible"],
    # Question 2
    ["Yes", "No", "It was okay"],
    # Question 3
    ["Red", "Blue", "Green", "Yellow", "Orange", "Purple", "Pink", "Black", "White", "Gray", "Brown", "Cyan", "Magenta", "Lime", "Teal", "Navy", "Other"],
    # Question 4
    ["Tea", "Coffee"],
    # Question 5
    ["Action", "Comedy", "Drama", "Sci-Fi", "Horror", "Romance", "Crime", "Documentary", "Animation", "War", "Thriller", "Biography", "Western", "History"]
]

In [45]:
def analyze_mood(questions, answers):
    selected_genres = []
    mood_score = 0
    
    for i, question in enumerate(questions):
        print(question)
        for j, option in enumerate(answers[i]):
            print(f"{j + 1}. {option}")
        selected_options = input("Select option(s) (comma-separated for multiple options): ").split(",")
        
        for selected_option in selected_options:
            try:
                option = answers[i][int(selected_option) - 1]
                sentiment_score = TextBlob(option).sentiment.polarity
                mood_score += sentiment_score
                
                if i == len(questions) - 1:
                    selected_genres.append(option)
            except (IndexError, ValueError):
                print("Invalid option selected.")
                continue
    
    mood_score = max(-1, min(1, mood_score))
    
    print(f"\nUser's mood score: {mood_score}")
    print(f"Selected genres: {selected_genres}")
    return mood_score, selected_genres


In [49]:
# Step 1: Load and preprocess the dataset
df = pd.read_csv('imdb_top_1000.csv')  
# Remove rows with missing overview
df = df.dropna(subset=['Overview']) 
# Convert 'Runtime' column to numeric values
df['Runtime'] = pd.to_numeric(df['Runtime'], errors='coerce')  

# Step 2: Perform sentiment analysis on the movie overviews
df['sentiment_score'] = df['Overview'].apply(lambda x: TextBlob(x).sentiment.polarity)

user_mood_score, selected_genres = analyze_mood(questions, answers)
print(f"\nUser's mood score: {user_mood_score}")
print(f"Selected genres: {selected_genres}")
# Step 4: Filter movies based on mood score
filtered_movies_mood = df[df['sentiment_score'] >= user_mood_score]

# Step 5: Filter movies based on selected genres
filtered_movies_genre = df[df['Genre'].str.contains('|'.join(selected_genres))]

# Step 6: Combine the filters using logical OR (|) to get the final recommended movies
recommended_movies = pd.concat([filtered_movies_mood, filtered_movies_genre], ignore_index=True).drop_duplicates()

# Step 7: Filter movies again based on selected genres (exact match)
recommended_movies = recommended_movies[recommended_movies['Genre'].str.contains('|'.join(selected_genres))]

# Step 8: Sort recommended movies based on sentiment scores
recommended_movies = recommended_movies.sort_values('sentiment_score', ascending=False).head(5)


How are you feeling today?
1. Great
2. Good
3. Neutral
4. Not so good
5. Terrible
Select option(s) (comma-separated for multiple options): 1
Did you enjoy your day?
1. Yes
2. No
3. It was okay
Select option(s) (comma-separated for multiple options): 1
What is your favorite color?
1. Red
2. Blue
3. Green
4. Yellow
5. Orange
6. Purple
7. Pink
8. Black
9. White
10. Gray
11. Brown
12. Cyan
13. Magenta
14. Lime
15. Teal
16. Navy
17. Other
Select option(s) (comma-separated for multiple options): 16
Do you prefer tea or coffee?
1. Tea
2. Coffee
Select option(s) (comma-separated for multiple options): 1
Which movie genres do you enjoy? (Select multiple options)
1. Action
2. Comedy
3. Drama
4. Sci-Fi
5. Horror
6. Romance
7. Crime
8. Documentary
9. Animation
10. War
11. Thriller
12. Biography
13. Western
14. History
Select option(s) (comma-separated for multiple options): 11,14

User's mood score: 0.8
Selected genres: ['Thriller', 'History']

User's mood score: 0.8
Selected genres: ['Thriller', 

In [ ]:
print("\nRecommended Movies:\n")
for index, movie in recommended_movies.iterrows():
    print(movie['Series_Title'] + " " + movie['Genre'])
    print("\n")